In [36]:
%run sandbox.ipynb

Loading credentials from token.json


In [37]:
main()

Anki sync successful.
Total cards found in 'Pleco Import': 1659
success
Latest flashcard xml last modified time: 2025-01-02 12:17:32 EST-0500
1648 flashcard entries found| 1 error entries found


In [108]:
import json

with open('flashcard_entries.json', 'r') as file:
    flashcard_entries = json.load(file)


In [102]:
flashcard_entries[1]

{'simplified': '心电感应',
 'traditional': '心電感應',
 'pinyin': 'xīn diàn gǎn yìng',
 'definition': 'telepathy',
 'dictid': 'PCED',
 'formatted_back': '<div align="left"><p><span style="font-size:32px";>心電感應〔心电感应〕</span><br>\n<span style="color:#B4B4B4;"><b><span style="font-size:0.80em;">PY </span></b></span><span style="color:#E30000;"><span style="font-weight:600;">xīn</span></span><span style="font-weight:600;"><span style="color:#8900BF;">diàn</span></span><span style="font-weight:600;"><span style="color:#1510F0;">gǎn</span></span><span style="font-weight:600;"><span style="color:#8900BF;">yìng</span></span></p>\n</div><div align="left"><p>telepathy</p>\n</div><plecoentry c="00000000" d="50434544" e="01d97500" x="-1"/>'}

In [103]:
from enum import Enum


class ToneColor(Enum):
    RED = "#E30000"
    GREEN = "#02B31C"
    PURPLE = "#8900BF"
    BLUE = "#1510F0"
    GREY = "#777777"  # Neutral tone


def get_pinyin_color(pinyin):
    tone_map = {
        "āēīōūǖ": ToneColor.RED,
        "áéíóúǘ": ToneColor.GREEN,
        "ǎěǐǒǔǚ": ToneColor.BLUE,
        "àèìòùǜ": ToneColor.PURPLE,
    }

    tone_color = ToneColor.GREY  # Default to neutral tone
    for chars, t in tone_map.items():
        if any(char in pinyin for char in chars):
            tone_color = t
            break

    return tone_color.value

In [104]:
def fmt_entry(entry):
    traditional = entry.get("traditional", "")
    simplified = entry.get("simplified", "")
    pinyin = entry.get("pinyin", "")
    definition = entry.get("definition", "")
    simplified_hint = f"〔{simplified}〕" if traditional != simplified else ""

    formatted_back = ""
    formatted_back += f'<div align="left"><p><span style="font-size:32px">{traditional}{simplified_hint}</span><br/>\n'
    formatted_back += '<span style="color:#B4B4B4;"><b><span style="font-size:0.80em;">PY </span></b></span>'
    for p in pinyin.split(" "):
        if "//" in p:
            p = p.replace("//", "")
            formatted_back += '<span style="font-weight:600;">//</span>'
        formatted_back += f'<span style="color:{get_pinyin_color(p)};"><span style="font-weight:600;">{p}</span></span>'

    return formatted_back

In [105]:
from bs4 import BeautifulSoup

def reorder_nested_spans(html_text):
    # Parse the HTML text
    soup = BeautifulSoup(f"<root>{html_text}</root>", 'html.parser')

    # Find all span elements
    spans = soup.find_all('span')

    for span in spans:
        style = span.get('style', '')
        if 'font-weight' in style:
            nested_span = span.find('span')
            if nested_span is not None:
                nested_style = nested_span.get('style', '')
                if 'color' in nested_style:
                    # Swap the styles
                    span['style'], nested_span['style'] = nested_span['style'], span['style']
        
        if ";" in spans[0].attrs:
            del spans[0].attrs[";"]

    # Convert the soup back to a string
    return str(soup).replace('<root>', '').replace('</root>', '')

# Example usage
html_text = '<span style="font-weight:600;"><span style="color:#FF0000;">Text</span></span>'
reordered_html = reorder_nested_spans(html_text)
print(reordered_html)


<span style="color:#FF0000;"><span style="font-weight:600;">Text</span></span>


In [113]:
def drop(arr, to_drop):
    arr = arr.copy()
    for i in to_drop:
        arr.pop(i)
    return arr

def grade_fmt_entry(flashcard_entries, n_error_char_show=10, to_drop=[]):
    flashcard_entries = drop(flashcard_entries, to_drop)
    correct_count = 0
    length_diff_count = 0
    for i, entry in enumerate(flashcard_entries):
        expected = entry['formatted_back']
        expected = expected.replace(' ;=""', ";")
        expected = reorder_nested_spans(expected)
        result = fmt_entry(entry)
        
        if expected != result:
            for j in range(min(len(expected), len(result))):
                if expected[j] != result[j]:
                    print(f"Entry {i} differs at character {j}:")
                    print(f"Exp: {repr(expected[j:j+n_error_char_show])}...")
                    print(f"Got: {repr(result[j:j+n_error_char_show])}...")
                    break
                    # return
            else:
                length_diff_count += 1
                j = len(result)
                # print(f"{i}: {repr(expected[j:j+n_error_char_show])}...")
                continue
        else:
            correct_count += 1
    print(f"Total correct entries: {correct_count}")
    print(f"Total entries differing only in length: {length_diff_count}")

In [122]:
to_drop = [752, 376, 51, 74, 85, 99, 112, 126, 131]
grade_fmt_entry(flashcard_entries, 1000, to_drop)

Entry 52 differs at character 319:
Exp: 'font-weight:600;"> </span><span style="color:#02B31C;"><span style="font-weight:600;">chén</span></span><span style="color:#8900BF;"><span style="font-weight:600;">mò</span></span></p>\n</div><div align="left"><p>remain silent</p>\n</div><plecoentry c="00000000" d="50414345" e="04083eaa" x="-1"></plecoentry>'...
Got: 'color:#02B31C;"><span style="font-weight:600;">chén</span></span><span style="color:#8900BF;"><span style="font-weight:600;">mò</span></span>'...
Entry 68 differs at character 327:
Exp: 'font-weight:600;"> </span><span style="color:#E30000;"><span style="font-weight:600;">guāng</span></span><span style="color:#02B31C;"><span style="font-weight:600;">lín</span></span></p>\n</div><div align="left"><p>welcome (to)</p>\n</div><plecoentry c="00000000" d="50414345" e="02881eb2" x="-1"></plecoentry>'...
Got: 'color:#E30000;"><span style="font-weight:600;">guāng</span></span><span style="color:#02B31C;"><span style="font-weight:600;">lín</

In [121]:
target = 131
drop(flashcard_entries, to_drop)[target]

{'simplified': '运动',
 'traditional': '運動',
 'pinyin': 'yùn dong',
 'definition': 'verb often pejorative arrange things through pull; use irregular means to gain ends; go lobbying 運動官府 yùndong guānfǔ canvass support from the authorities',
 'dictid': 'PACE',
 'formatted_back': '<div align="left"><p><span style="font-size:32px";>運動〔运动〕</span><br>\n<span style="color:#B4B4B4;"><b><span style="font-size:0.80em;">PY </span></b></span><span style="color:#8900BF;"><span style="font-weight:600;">yùn</span></span><span style="font-weight:600;"><span style="color:#8900BF;">dòng</span></span></p>\n</div><div align="left"><p><b><span style="font-size:0.80em;"><span style="color:#B4B4B4;">VERB</span></span></b><br>\n<b>1\t</b>motion; movement<br>\n</p>\n<blockquote style="border-left: 2px solid #0078c3; margin-left: 3px; padding-left: 1em; margin-top: 0px; margin-bottom: 0px;"><p><b><span style="color:#0078C3;">運動</span></b><span style="color:#0078C3;">的形式</span><br>\n<b>yùndòng</b><span style="font

In [65]:
expected = flashcard_entries[target]['formatted_back']
# expected = expected.replace(' ;=""', ";")
expected = reorder_nested_spans(expected)
print(expected)

<div align="left"><p><span ;="" style="font-size:32px">艘</span><br/>
<span style="color:#B4B4B4;"><b><span style="font-size:0.80em;">PY </span></b></span><span style="color:#E30000;"><span style="font-weight:600;">sōu</span></span></p>
</div><div align="left"><p><b><span style="font-size:0.80em;"><span style="color:#B4B4B4;">MEASURE WORD</span></span></b><br/>
[for boats or ships]<br/>
</p>
<blockquote style="border-left: 2px solid #0078c3; margin-left: 3px; padding-left: 1em; margin-top: 0px; margin-bottom: 0px;"><p><span style="color:#0078C3;">兩</span><span style="color:#0078C3;"><b>艘</b></span><span style="color:#0078C3;">油船</span><br/>
<span style="font-weight:600;">Liǎng </span><b>sōu</b><span style="font-weight:600;"> yóuchuán</span><br/>
two tankers<br/>
</p>
</blockquote>
<blockquote style="border-left: 2px solid #0078c3; margin-left: 3px; padding-left: 1em; margin-top: 0px; margin-bottom: 0px;"><p><span style="color:#0078C3;">一</span><span style="color:#0078C3;"><b>艘</b></span

In [66]:
# Parse the HTML text
soup = BeautifulSoup('<root><span style="font-size:32px";>hi</span></root>', 'html.parser')

# Find all span elements
spans = soup.find_all('span')
spans[0]


<span ;="" style="font-size:32px">hi</span>

In [68]:
if ";" in spans[0].attrs:
    del spans[0].attrs[";"]

In [69]:
spans[0]

<span style="font-size:32px">hi</span>